# Supercharging Your Web Development with FastAPI

Welcome to the exciting world of FastAPI, where web development meets speed and simplicity! In this live Jupyter demonstration, we'll delve into the ins and outs of FastAPI, a modern Python web framework that's been gaining traction for its lightning-fast performance and intuitive API design.

Whether you're a seasoned developer looking to streamline your workflow or a newcomer eager to explore the latest tools in Python web development, this session is tailor-made for you. Join us as we explore FastAPI's key features, walk through code examples, and demonstrate how it can revolutionize the way you build web applications - all within the interactive environment of Jupyter.

Get ready to supercharge your development process with FastAPI - because in the fast-paced world of web development, every millisecond counts!

May 16, 2024

By baronpythonist (Byron): Intro generated from OpenAI's ChatGPT.

## Imports and app instantiation

In [79]:
from fastapi import FastAPI, status, HTTPException
from asyncio import sleep
import nest_asyncio
from pydantic import BaseModel, datetime_parse
from typing import Optional, Union
from enum import Enum
import uvicorn
from uuid import uuid4

app = FastAPI()

## Example Pydantic Models

In [80]:
class PyTalk(BaseModel):
    id: str | None = None
    datetime: str
    topic: str
    speaker: str
    description: str | None = None

class EventType(str, Enum):
    meal = "meal"
    main = "main"
    keynote = "keynote"
    lightning = "lightning"

class PyEvent(BaseModel):
    id: str | None = None
    datetime: str
    title: str
    type: EventType

class PySchedule(BaseModel):
    id: str | None = None
    name: str
    talks: list[PyTalk] = []
    events: list[PyEvent] = []

## Example Endpoints

In [81]:
class DataBase:
    schedules: dict[str,PySchedule] = {}

@app.get('/')
def info_dump():
    """info_dump(): Simple hello world endpoint for PyCon.
    
    params: None
    output:
        JSON object:
            message: `str` = hello-world-style message
    """
    return {'message': 'Hello PyCon US 2024!'}

@app.post('/schedule', status_code=status.HTTP_201_CREATED)
def create_schedule(schedule: PySchedule):
    """create_schedule(schedule): Creates a schedule from a validated JSON object
    
    params:
        schedule: `PySchedule` = conference schedule defined by Pydantic model
    output:
        schedule: `PySchedule` = schedule + id
    """
    if schedule.id is None:
        schedule.id = str(uuid4())
        DataBase.schedules[schedule.id] = schedule
        return schedule
    else:
        raise HTTPException(status_code=status.HTTP_409_CONFLICT, detail="Schedule already exists!")

@app.get('/schedule/talks')
def get_all_talks():
    """get_all_talks():
    
    params: None
    output:
        JSON object:
            all_talks: `list[PyTalk]` = list of talks with non-empty ids
    """
    all_talks = []
    for _, schedule in DataBase.schedules.items():
        all_talks.extend([talk for talk in schedule.talks if talk.id is not None])
    return {"all_talks": all_talks}

@app.get('/schedule/{sch_id}/talks/{talk_id}')
def get_talk(sch_id: str, talk_id: str):
    """get_talk(sch_id, talk_id): Get specific talk by ID.
    
    params:
        sch_id: `str` = uuid of `PySchedule` as `str`
        talk_id: `str` = uuid of `PyTalk` as `str`
    output:

    """
    if sch_id in DataBase.schedules:
        for talk in DataBase.schedules[sch_id].talks:
            if talk.id == talk_id:
                return {"talk": talk}
    raise HTTPException(status_code=status.HTTP_404_NOT_FOUND, detail="talk not found")

## Example API launcher

In [84]:
def launcher(app):
    def launch_app():
        nest_asyncio.apply()
        uvicorn.run(app)
    return launch_app

Spin-up API; interupt this cell to take down API and continue (ignore `KeyboardInterruptError` message).

## Example Shedule Body (Copy into Swagger)

```json
{
 "name": "PyConUS24",
  "talks": [
    {
      "id": "01",
      "datetime": "Friday @ 11:00 AM",
      "topic": "FastAPI Discussion and Demo",
      "speaker": "Byron Burks",
      "description": null
    }
  ],
  "events": [
    {
      "id": "11",
      "datetime": "Saturday @ 7:30 AM",
      "title": "Breakfast",
      "type": "meal"
    }
  ]
}
```

In [ ]:
runf = launcher(app)
runf()

## Playground area: write new API endpoints and try them out!